In [127]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_**** = 'https://s3.eu-geo.objectstorage.softlayer.net'
else:
    endpoint_***** = 'https://s3.eu-geo.objectstorage.service.networklayer.com'

client_***** = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='******',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_04abeae85fc94048947b8ed7bfe090d5)

body = client_****.get_object(Bucket='capstoneproject-donotdelete-pr-egev5iahubzuav',Key='Zurich_districts_geo.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_2 = pd.read_csv(body)
df_data_2.head()


,District,Neighborhood,Latitude,Longitude
0,District 1,"Rathaus, Hochschulen, Lindenhof, City",47.3742,8.5396
1,District 2,"Wollishofen, Leimbach , Enge",47.3441,8.5300
2,District 3,"Alt-Wiedikon, Friesenberg, Sihlfeld",47.3651,8.5026
3,District 4,"Werd, Langstrasse, Hard",47.3801,8.5207
4,District 5,"Gewerbeschule, Escher Wyss, Unterstrass, Obers...",47.3873,8.5190


Import Libraries

In [128]:
!pip install beautifulsoup4
import requests
import bs4 as BeautifulSoup
import pandas as pd

In [129]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


### Import Folium

In [130]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.7.0 --yes 
import folium # map rendering library

Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/noarch::ibm-wsrt-py37main-main==custom=1937
  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37hafa7651_0
  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=1937
done

# All requested packages already installed.

Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/noarch::ibm-wsrt-py37main-main==custom=1937
  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37hafa7651_0
  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=1937
done

# All requested packages already installed.



Get the latitude and longitude values of Zurich using Geopy

In [131]:
address = 'Zürich, CH'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Zürich are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Zürich are 47.3744489, 8.5410422.


Create a map of Zürich with District superimposed on top. Use latitude and logitude

In [132]:
map_zurich = folium.Map(location=[latitude, longitude], zoom_start=9.5)

# add markers to map
for lat, lng, borough in zip(df_data_1['Latitude'], df_data_1['Longitude'], df_data_1['District']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_zurich)  
    
map_zurich

### Let's use the Foursquare API

In [133]:
CLIENT_ID = 'AF3NMAQEM3DHBZQICGWEJVIO0T1NAOCZSHGD3NMMAS1NO0CM'
CLIENT_SECRET = '4GPVXUK0EGHNE5GABYKY3V3PML5EWL4G0INMT0R0XB0RTT3J'
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

Get the district's latitude and longitude values.

In [134]:
zurich_data = df_data_1

borough_latitude = zurich_data.loc[0, 'Latitude'] # neighborhood latitude value
borough_longitude = zurich_data.loc[0, 'Longitude'] # neighborhood longitude value

borough_name = zurich_data.loc[0, 'District'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(borough_name, 
                                                               borough_latitude, 
                                                               borough_longitude))

Latitude and longitude values of District 1 are 47.3742, 8.5396.


In [135]:
LIMIT = 100
radius = 750
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    borough_latitude, 
    borough_longitude, 
    radius, 
    LIMIT)


In [136]:
! pip install requests

In [137]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '604fbb37c6a9514363db015f'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Lindenhof',
  'headerFullLocation': 'Lindenhof, Zürich',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 189,
  'suggestedBounds': {'ne': {'lat': 47.380950006750005,
    'lng': 8.549548804597537},
   'sw': {'lat': 47.36744999325, 'lng': 8.529651195402463}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b042417f964a5208f5122e3',
       'name': 'Lindenhof',
       'location': {'address': 'Lindenhof',
        'lat': 47.373004877169656,
        'lng': 8.54088306427002,
        'labeledLatLngs': [{'label': 'display',
          'lat': 47.373004877169656,
  

In [138]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [139]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


AttributeError: 'Series' object has no attribute '_mgr'

In [140]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [141]:
nearby_venues

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Lindenhof,"[{'id': '52e81612bcbc57f1066b7a25', 'name': 'P...",47.373005,8.540883
1,Globus Delicatessa,"[{'id': '4bf58dd8d48988d1f5941735', 'name': 'G...",47.375557,8.538357
2,Widder Bar,"[{'id': '4bf58dd8d48988d1d5941735', 'name': 'H...",47.372415,8.539863
3,Dachterrasse Hiltl,"[{'id': '4bf58dd8d48988d1d3941735', 'name': 'V...",47.375686,8.539650
4,Widder Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",47.372449,8.539973
5,Bahnhofstrasse,"[{'id': '52e81612bcbc57f1066b7a25', 'name': 'P...",47.374699,8.538872
6,Rooftop Bar,"[{'id': '4bf58dd8d48988d121941735', 'name': 'L...",47.375556,8.540126
7,Old Crow,"[{'id': '4bf58dd8d48988d11e941735', 'name': 'C...",47.372092,8.541024
8,Yumi Hana Lebensmittel & Spezialitäten,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",47.376343,8.540386
9,Hiltl,"[{'id': '4bf58dd8d48988d1d3941735', 'name': 'V...",47.373173,8.536831


Create a funtion to get all the district in Zurich

In [142]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [143]:
Zurich_venues = getNearbyVenues(names=zurich_data['District'],
                                   latitudes=zurich_data['Latitude'],
                                   longitudes=zurich_data['Longitude']
                                  )

District 1
District 2
District 3
District 4
District 5
District 6
District 7
District 8
District 9
District 10
District 11
District 12


In [144]:
print(Zurich_venues.shape)
Zurich_venues.head()

(679, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,District 1,47.3742,8.5396,Lindenhof,47.373005,8.540883,Pedestrian Plaza
1,District 1,47.3742,8.5396,Globus Delicatessa,47.375557,8.538357,Gourmet Shop
2,District 1,47.3742,8.5396,Widder Bar,47.372415,8.539863,Hotel Bar
3,District 1,47.3742,8.5396,Dachterrasse Hiltl,47.375686,8.539650,Vegetarian / Vegan Restaurant
4,District 1,47.3742,8.5396,Widder Hotel,47.372449,8.539973,Hotel


In [145]:
Zurich_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
District 1,100,100,100,100,100,100
District 10,30,30,30,30,30,30
District 11,56,56,56,56,56,56
District 12,21,21,21,21,21,21
District 2,30,30,30,30,30,30
District 3,41,41,41,41,41,41
District 4,100,100,100,100,100,100
District 5,100,100,100,100,100,100
District 6,54,54,54,54,54,54


In [146]:
print('There are {} uniques categories.'.format(len(Zurich_venues['Venue Category'].unique())))

There are 169 uniques categories.


### Analyzing Each Borough

In [147]:
# one hot encoding
Zurich_onehot = pd.get_dummies(Zurich_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Zurich_onehot['Neighborhood'] = Zurich_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Zurich_onehot.columns[-1]] + list(Zurich_onehot.columns[:-1])
Zurich_onehot = Zurich_onehot[fixed_columns]

Zurich_onehot.head()

,Neighborhood,Accessories Store,American Restaurant,Argentinian Restaurant,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Dealership,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bar,Bathing Area,Beach,Beer Garden,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Breakfast Spot,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Cable Car,Cafeteria,Café,Candy Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Community Center,Concert Hall,Convenience Store,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Dance Studio,Department Store,Dessert Shop,Diner,Discount Store,Doner Restaurant,Electronics Store,Ethiopian Restaurant,Exhibit,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hill,Hockey Rink,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Island,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Meyhane,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Museum,Music Venue,Nightclub,Office,Other Great Outdoors,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pier,Pizza Place,Platform,Plaza,Pool,Pool Hall,Print Shop,Pub,Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Shoe Store,Shopping Mall,Skating Rink,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Swiss Restaurant,Taverna,Tennis Court,Thai Restaurant,Theater,Tibetan Restaurant,Train Station,Tram Station,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Watch Shop,Wine Bar,Wine Shop,Zoo,Zoo Exhibit
0,District 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,District 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,District 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,District 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,District 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [148]:
Zurich_onehot.shape

(679, 170)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [149]:
Zurich_grouped = Zurich_onehot.groupby('Neighborhood').mean().reset_index()
Zurich_grouped

,Neighborhood,Accessories Store,American Restaurant,Argentinian Restaurant,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Dealership,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bar,Bathing Area,Beach,Beer Garden,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Breakfast Spot,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Cable Car,Cafeteria,Café,Candy Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Community Center,Concert Hall,Convenience Store,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Dance Studio,Department Store,Dessert Shop,Diner,Discount Store,Doner Restaurant,Electronics Store,Ethiopian Restaurant,Exhibit,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hill,Hockey Rink,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Island,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Meyhane,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Museum,Music Venue,Nightclub,Office,Other Great Outdoors,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pier,Pizza Place,Platform,Plaza,Pool,Pool Hall,Print Shop,Pub,Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Shoe Store,Shopping Mall,Skating Rink,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Swiss Restaurant,Taverna,Tennis Court,Thai Restaurant,Theater,Tibetan Restaurant,Train Station,Tram Station,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Watch Shop,Wine Bar,Wine Shop,Zoo,Zoo Exhibit
0,District 1,0.00,0.000000,0.010000,0.01,0.02,0.000000,0.000000,0.00,0.000000,0.000000,0.030000,0.090000,0.000000,0.000000,0.00000,0.010000,0.01,0.010000,0.000000,0.02,0.00000,0.000000,0.000000,0.01,0.000000,0.000000,0.00,0.000000,0.000000,0.070000,0.01,0.000000,0.010000,0.01,0.01,0.06,0.020000,0.000000,0.000000,0.000000,0.00,0.01,0.00,0.02,0.000000,0.04,0.03,0.01,0.000000,0.000000,0.00,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.00,0.010000,0.00,0.01,0.000000,0.000000,0.000000,0.000000,0.020000,0.010000,0.010000,0.020000,0.000000,0.000000,0.000000,0.000000,0.030000,0.010000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.01,0.01,0.000000,0.000000,0.00,0.000000,0.01,0.020000,0.00,0.000000,0.010000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.02,0.000000,0.00,0.000000,0.000000,0.000000,0.040000,0.010000,0.00,0.000000,0.000000,0.030000,0.00,0.000000,0.000000,0.00,0.000000,0.01,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.010000,0.010000,0.00,0.000000,0.000000,0.000000,0.010000,0.00,0.010000,0.040000,0.000000,0.000000,0.020000,0.000000,0.00,0.010000,0.000000,0.000000,0.00,0.040000,0.00000,0.000000,0.01,0.000000,0.010000,0.000000,0.000000
1,District 10,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.00,0.00000,0.000000,0.000000,0.00,0.200000,0.000000,0.00,0.000000,0.000000,0.033333,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.033333,0.000000,0.00,0.00,0.00,0

In [150]:
Zurich_grouped.shape

(12, 170)

In [151]:
Zurich_grouped

,Neighborhood,Accessories Store,American Restaurant,Argentinian Restaurant,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Dealership,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bar,Bathing Area,Beach,Beer Garden,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Breakfast Spot,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Cable Car,Cafeteria,Café,Candy Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Community Center,Concert Hall,Convenience Store,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Dance Studio,Department Store,Dessert Shop,Diner,Discount Store,Doner Restaurant,Electronics Store,Ethiopian Restaurant,Exhibit,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hill,Hockey Rink,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Island,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Meyhane,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Museum,Music Venue,Nightclub,Office,Other Great Outdoors,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pier,Pizza Place,Platform,Plaza,Pool,Pool Hall,Print Shop,Pub,Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Shoe Store,Shopping Mall,Skating Rink,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Swiss Restaurant,Taverna,Tennis Court,Thai Restaurant,Theater,Tibetan Restaurant,Train Station,Tram Station,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Watch Shop,Wine Bar,Wine Shop,Zoo,Zoo Exhibit
0,District 1,0.00,0.000000,0.010000,0.01,0.02,0.000000,0.000000,0.00,0.000000,0.000000,0.030000,0.090000,0.000000,0.000000,0.00000,0.010000,0.01,0.010000,0.000000,0.02,0.00000,0.000000,0.000000,0.01,0.000000,0.000000,0.00,0.000000,0.000000,0.070000,0.01,0.000000,0.010000,0.01,0.01,0.06,0.020000,0.000000,0.000000,0.000000,0.00,0.01,0.00,0.02,0.000000,0.04,0.03,0.01,0.000000,0.000000,0.00,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.00,0.010000,0.00,0.01,0.000000,0.000000,0.000000,0.000000,0.020000,0.010000,0.010000,0.020000,0.000000,0.000000,0.000000,0.000000,0.030000,0.010000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.01,0.01,0.000000,0.000000,0.00,0.000000,0.01,0.020000,0.00,0.000000,0.010000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.02,0.000000,0.00,0.000000,0.000000,0.000000,0.040000,0.010000,0.00,0.000000,0.000000,0.030000,0.00,0.000000,0.000000,0.00,0.000000,0.01,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.010000,0.010000,0.00,0.000000,0.000000,0.000000,0.010000,0.00,0.010000,0.040000,0.000000,0.000000,0.020000,0.000000,0.00,0.010000,0.000000,0.000000,0.00,0.040000,0.00000,0.000000,0.01,0.000000,0.010000,0.000000,0.000000
1,District 10,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.00,0.00000,0.000000,0.000000,0.00,0.200000,0.000000,0.00,0.000000,0.000000,0.033333,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.033333,0.000000,0.00,0.00,0.00,0

Let's put that into a pandas dataframe and extract only Restaurants

In [152]:
print(Zurich_grouped. columns)

Index(['Neighborhood', 'Accessories Store', 'American Restaurant',
       'Argentinian Restaurant', 'Art Museum', 'Arts & Crafts Store',
       'Asian Restaurant', 'Auto Dealership', 'Automotive Shop', 'BBQ Joint',
       ...
       'Tunnel', 'Turkish Restaurant', 'Vegetarian / Vegan Restaurant',
       'Video Store', 'Vietnamese Restaurant', 'Watch Shop', 'Wine Bar',
       'Wine Shop', 'Zoo', 'Zoo Exhibit'],
      dtype='object', length=170)


In [153]:
New_zurich_grouped=Zurich_grouped[['Neighborhood', 'American Restaurant', 'Argentinian Restaurant', 'Asian Restaurant', 'Chinese Restaurant', 'Cuban Restaurant', 'Doner Restaurant', 'Ethiopian Restaurant', 'Falafel Restaurant', 'Fast Food Restaurant', 'Fondue Restaurant', 'French Restaurant', 'Indian Restaurant', 'Indonesian Restaurant', 'Italian Restaurant', 'Japanese Restaurant', 'Kebab Restaurant', 'Korean Restaurant', 'Mediterranean Restaurant', 'Mexican Restaurant', 'Middle Eastern Restaurant', 'Modern European Restaurant', 'Molecular Gastronomy Restaurant', 'Moroccan Restaurant', 'Paella Restaurant', 'Peruvian Restaurant', 'Restaurant', 'South American Restaurant', 'Southern / Soul Food Restaurant', 'Spanish Restaurant', 'Sushi Restaurant', 'Swiss Restaurant', 'Thai Restaurant', 'Tibetan Restaurant', 'Turkish Restaurant', 'Vegetarian / Vegan Restaurant', 'Vietnamese Restaurant']].copy()

In [154]:
New_zurich_grouped

,Neighborhood,American Restaurant,Argentinian Restaurant,Asian Restaurant,Chinese Restaurant,Cuban Restaurant,Doner Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Fondue Restaurant,French Restaurant,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Korean Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,Paella Restaurant,Peruvian Restaurant,Restaurant,South American Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sushi Restaurant,Swiss Restaurant,Thai Restaurant,Tibetan Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,District 1,0.000000,0.010000,0.000000,0.010000,0.01,0.000000,0.00000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.030000,0.000000,0.00,0.010000,0.010000,0.040000,0.020000,0.00,0.00,0.040000,0.000000
1,District 10,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.00,0.000000,0.033333,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000
2,District 11,0.017857,0.017857,0.000000,0.017857,0.00,0.000000,0.00000,0.017857,0.017857,0.000000,0.000000,0.035714,0.017857,0.053571,0.000000,0.035714,0.00,0.000000,0.000000,0.017857,0.000000,0.000000,0.00,0.000000,0.00,0.053571,0.000000,0.00,0.000000,0.017857,0.035714,0.035714,0.00,0.00,0.017857,0.035714
3,District 12,0.000000,0.000000,0.047619,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.047619,0.000000,0.00,0.000000,0.000000,0.095238,0.047619,0.00,0.00,0.000000,0.000000
4,District 2,0.033333,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.033333,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.066667,0.000000,0.00,0.000000,0.000000,0.033333,0.000000,0.00,0.00,0.000000,0.000000
5,District 3,0.000000,0.000000,0.048780,0.000000,0.00,0.000000,0.02439,0.024390,0.000000,0.000000,0.000000,0.000000,0.000000,0.024390,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.024390,0.00,0.000000,0.00,0.024390,0.000000,0.00,0.000000,0.000000,0.024390,0.024390,0.00,0.00,0.000000,0.000000
6,District 4,0.000000,0.000000,0.040000,0.030000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.060000,0.040000,0.000000,0.00,0.010000,0.000000,0.030000,0.000000,0.000000,0.01,0.000000,0.01,0.010000,0.000000,0.00,0.000000,0.010000,0.050000,0.040000,0.01,0.00,0.010000,0.010000
7,District 5,0.000000,0.000000,0.020000,0.000000,0.00,0.000000,0.00000,0.000000,0.010000,0.000000,0.000000,0.010000,0.000000,0.050000,0.030000,0.000000,0.01,0.010000,0.000000,0.010000,0.010000,0.000000,0.01,0.000000,0.01,0.020000,0.000000,0.01,0.020000,0.000000,0.030000,0.010000,0.00,0.01,0.010000,0.010000
8,District 6,0.000000,0.000000,0.037037,0.000000,0.00,0.018519,0.00000,0.018519,0.000000,0.000000,0.000000,0.018519,0.000000,0.037037,0.000000,0.000000,0.00,0.000000,0.000000,0.037037,0.000000,0.000000,0.00,0.018519,0.00,0.018519,0.000000,0.00,0.000000,0.000000,0.037037,0.018519,0.00,0.00,0.000000,0.000000
9,District 7,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.034483,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.034483,0.00,0.000000,0.00,0.103448,0.034483,0.00,0.000000,0.000000,0.068966,0.000000,0.00,0.00,0.000000,0.000000


let's write a function to sort the venues in descending order.

In [155]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

let's create the new dataframe and display the top 10 venues for each neighborhood.

In [156]:
num_top_venues = 10

for hood in New_zurich_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = New_zurich_grouped[New_zurich_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    

----District 1----
                           venue  freq
0  Vegetarian / Vegan Restaurant  0.04
1               Swiss Restaurant  0.04
2                     Restaurant  0.03
3                Thai Restaurant  0.02
4              French Restaurant  0.01
5             Chinese Restaurant  0.01
6               Cuban Restaurant  0.01
7         Argentinian Restaurant  0.01
8       Mediterranean Restaurant  0.01
9               Sushi Restaurant  0.01
----District 10----
                             venue  freq
0               Italian Restaurant  0.07
1               Mexican Restaurant  0.03
2             Fast Food Restaurant  0.03
3  Southern / Soul Food Restaurant  0.00
4  Molecular Gastronomy Restaurant  0.00
5              Moroccan Restaurant  0.00
6                Paella Restaurant  0.00
7              Peruvian Restaurant  0.00
8                       Restaurant  0.00
9        South American Restaurant  0.00
----District 11----
                           venue  freq
0                     

### Let's put that into a pandas dataframe

First, let's write a function to sort the venues in descending order.

In [157]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 5 venues for each neighborhood.

In [158]:
import numpy as np

In [191]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = New_zurich_grouped['Neighborhood']

for ind in np.arange(New_zurich_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(New_zurich_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,District 1,Vegetarian / Vegan Restaurant,Swiss Restaurant,Restaurant,Thai Restaurant,Chinese Restaurant,French Restaurant,Cuban Restaurant,Mediterranean Restaurant,Spanish Restaurant,Sushi Restaurant
1,District 10,Italian Restaurant,Fast Food Restaurant,Mexican Restaurant,Asian Restaurant,Chinese Restaurant,Cuban Restaurant,Doner Restaurant,Ethiopian Restaurant,Falafel Restaurant,Korean Restaurant
2,District 11,Restaurant,Italian Restaurant,Vietnamese Restaurant,Thai Restaurant,Swiss Restaurant,Indian Restaurant,Kebab Restaurant,Middle Eastern Restaurant,Argentinian Restaurant,Chinese Restaurant
3,District 12,Swiss Restaurant,Italian Restaurant,Asian Restaurant,Thai Restaurant,Restaurant,Vietnamese Restaurant,Fast Food Restaurant,Indonesian Restaurant,Indian Restaurant,French Restaurant
4,District 2,Restaurant,Mediterranean Restaurant,Swiss Restaurant,American Restaurant,Thai Restaurant,Tibetan Restaurant,Argentinian Restaurant,Asian Restaurant,Chinese Restaurant,Cuban Restaurant


### Cluster

Run k-means to cluster the neighborhood into 5 clusters.

In [200]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

zurich_grouped_clustering = New_zurich_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(zurich_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 1, 0, 3, 4, 1, 1, 1, 1, 2], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 5 venues for each neighborhood.

In [186]:
zurich_data

,District,Neighborhood,Latitude,Longitude
0,District 1,"Rathaus, Hochschulen, Lindenhof, City",47.3742,8.5396
1,District 2,"Wollishofen, Leimbach , Enge",47.3441,8.5300
2,District 3,"Alt-Wiedikon, Friesenberg, Sihlfeld",47.3651,8.5026
3,District 4,"Werd, Langstrasse, Hard",47.3801,8.5207
4,District 5,"Gewerbeschule, Escher Wyss, Unterstrass, Obers...",47.3873,8.5190
5,District 6,"Unterstrass, Oberstrass",47.3939,8.5449
6,District 7,"Fluntern, Hottingen, Hirslanden, Witikon",47.3776,8.5742
7,District 8,"Seefeld, Mhlebach, Weinegg",47.3575,8.5650
8,District 9,"Albisrieden, Altstetten",47.3860,8.4819
9,District 10,"Hoengg, Wipkingen",47.4024,8.4993


In [194]:
zurich_data2=zurich_data.rename(columns={"District": "Neighborhood","Neighborhood": "City parts"})

In [201]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

zurich_merged = zurich_data2

# merge zurich_grouped with zurich_data to add latitude/longitude for each neighborhood
zurich_merged = zurich_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
##neighborhoods_venues_sorted.set_index('Neighborhood')

zurich_merged.head() # check the last columns!

,Neighborhood,City parts,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,District 1,"Rathaus, Hochschulen, Lindenhof, City",47.3742,8.5396,4,Vegetarian / Vegan Restaurant,Swiss Restaurant,Restaurant,Thai Restaurant,Chinese Restaurant,French Restaurant,Cuban Restaurant,Mediterranean Restaurant,Spanish Restaurant,Sushi Restaurant
1,District 2,"Wollishofen, Leimbach , Enge",47.3441,8.5300,4,Restaurant,Mediterranean Restaurant,Swiss Restaurant,American Restaurant,Thai Restaurant,Tibetan Restaurant,Argentinian Restaurant,Asian Restaurant,Chinese Restaurant,Cuban Restaurant
2,District 3,"Alt-Wiedikon, Friesenberg, Sihlfeld",47.3651,8.5026,1,Asian Restaurant,Restaurant,Italian Restaurant,Thai Restaurant,Swiss Restaurant,Ethiopian Restaurant,Molecular Gastronomy Restaurant,Falafel Restaurant,Fondue Restaurant,Indonesian Restaurant
3,District 4,"Werd, Langstrasse, Hard",47.3801,8.5207,1,Italian Restaurant,Swiss Restaurant,Asian Restaurant,Thai Restaurant,Japanese Restaurant,Chinese Restaurant,Middle Eastern Restaurant,Vietnamese Restaurant,Moroccan Restaurant,Indian Restaurant
4,District 5,"Gewerbeschule, Escher Wyss, Unterstrass, Obers...",47.3873,8.5190,1,Italian Restaurant,Swiss Restaurant,Japanese Restaurant,Asian Restaurant,Spanish Restaurant,Restaurant,Vietnamese Restaurant,Modern European Restaurant,Fast Food Restaurant,Indian Restaurant


Finally, let's visualize the resulting clusters

In [202]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(zurich_merged['Latitude'], zurich_merged['Longitude'], zurich_merged['Neighborhood'], zurich_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

Now, I examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster.


Cluster 1

In [203]:
zurich_merged.loc[zurich_merged['Cluster Labels'] == 0, zurich_merged.columns[[1] + list(range(5, zurich_merged.shape[1]))]]

,City parts,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,"Affoltern, Oerlikon, Seebach",Restaurant,Italian Restaurant,Vietnamese Restaurant,Thai Restaurant,Swiss Restaurant,Indian Restaurant,Kebab Restaurant,Middle Eastern Restaurant,Argentinian Restaurant,Chinese Restaurant


Cluster 2

In [204]:
zurich_merged.loc[zurich_merged['Cluster Labels'] == 1, zurich_merged.columns[[1] + list(range(5, zurich_merged.shape[1]))]]

,City parts,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Alt-Wiedikon, Friesenberg, Sihlfeld",Asian Restaurant,Restaurant,Italian Restaurant,Thai Restaurant,Swiss Restaurant,Ethiopian Restaurant,Molecular Gastronomy Restaurant,Falafel Restaurant,Fondue Restaurant,Indonesian Restaurant
3,"Werd, Langstrasse, Hard",Italian Restaurant,Swiss Restaurant,Asian Restaurant,Thai Restaurant,Japanese Restaurant,Chinese Restaurant,Middle Eastern Restaurant,Vietnamese Restaurant,Moroccan Restaurant,Indian Restaurant
4,"Gewerbeschule, Escher Wyss, Unterstrass, Obers...",Italian Restaurant,Swiss Restaurant,Japanese Restaurant,Asian Restaurant,Spanish Restaurant,Restaurant,Vietnamese Restaurant,Modern European Restaurant,Fast Food Restaurant,Indian Restaurant
5,"Unterstrass, Oberstrass",Italian Restaurant,Asian Restaurant,Middle Eastern Restaurant,Swiss Restaurant,Doner Restaurant,Indian Restaurant,Paella Restaurant,Falafel Restaurant,Restaurant,Thai Restaurant
8,"Albisrieden, Altstetten",Italian Restaurant,Thai Restaurant,Swiss Restaurant,Japanese Restaurant,Mexican Restaurant,Middle Eastern Restaurant,French Restaurant,Fast Food Restaurant,Mediterranean Restaurant,Spanish Restaurant
9,"Hoengg, Wipkingen",Italian Restaurant,Fast Food Restaurant,Mexican Restaurant,Asian Restaurant,Chinese Restaurant,Cuban Restaurant,Doner Restaurant,Ethiopian Restaurant,Falafel Restaurant,Korean Restaurant


Cluster 3

In [205]:
zurich_merged.loc[zurich_merged['Cluster Labels'] == 2, zurich_merged.columns[[1] + list(range(5, zurich_merged.shape[1]))]]

,City parts,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,"Fluntern, Hottingen, Hirslanden, Witikon",Restaurant,Swiss Restaurant,Italian Restaurant,Molecular Gastronomy Restaurant,Fondue Restaurant,South American Restaurant,Falafel Restaurant,Indonesian Restaurant,Indian Restaurant,French Restaurant


Cluster 4

In [206]:
zurich_merged.loc[zurich_merged['Cluster Labels'] == 3, zurich_merged.columns[[1] + list(range(5, zurich_merged.shape[1]))]]

,City parts,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,"Seefeld, Mhlebach, Weinegg",Swiss Restaurant,Italian Restaurant,Restaurant,French Restaurant,Mediterranean Restaurant,Sushi Restaurant,Mexican Restaurant,Indian Restaurant,Modern European Restaurant,Fast Food Restaurant
11,"Saatlen, Schwamendingen Mitte, Hirzenbach",Swiss Restaurant,Italian Restaurant,Asian Restaurant,Thai Restaurant,Restaurant,Vietnamese Restaurant,Fast Food Restaurant,Indonesian Restaurant,Indian Restaurant,French Restaurant


Cluster 5

In [207]:
zurich_merged.loc[zurich_merged['Cluster Labels'] == 4, zurich_merged.columns[[1] + list(range(5, zurich_merged.shape[1]))]]

,City parts,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Rathaus, Hochschulen, Lindenhof, City",Vegetarian / Vegan Restaurant,Swiss Restaurant,Restaurant,Thai Restaurant,Chinese Restaurant,French Restaurant,Cuban Restaurant,Mediterranean Restaurant,Spanish Restaurant,Sushi Restaurant
1,"Wollishofen, Leimbach , Enge",Restaurant,Mediterranean Restaurant,Swiss Restaurant,American Restaurant,Thai Restaurant,Tibetan Restaurant,Argentinian Restaurant,Asian Restaurant,Chinese Restaurant,Cuban Restaurant
